In [1]:
!pip install "git+https://github.com/facebookresearch/pytorch3d.git"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/pytorch3d.git to /tmp/pip-req-build-ccmv4qcx
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-ccmv4qcx
     |████████████████████████████████| 50 kB 4.9 MB/s 
     |████████████████████████████████| 596 kB 26.4 MB/s 
  Created wheel for pytorch3d: filename=pytorch3d-0.6.2-cp37-cp37m-linux_x86_64.whl size=31527418 sha256=ccf1093db9c79457040e19f7e2e679b6bb3ff1eb8694f135c10c3467f2e0256c
  Stored in directory: /tmp/pip-ephem-wheel-cache-zmnio03_/wheels/db/2f/07/b84807ee4c9ffc917b90b716977d7fea8f9e841f6173150600
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=801797b046a6f68d924ee2c966695fb62e11880fd6273e4e69bee1de206319fa
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built py

In [2]:
import os
import numpy as np
import json

from PIL import Image  
from csv import writer
import csv

from tensorflow import keras
import tensorflow as tf

from typing import Tuple

# import torch
# import torch.nn.functional as F
# from pytorch3d import _C
# from torch.autograd import Function

from pytorch3d.ops import box3d_overlap
import torch

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jun 26 15:05:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
WIDTH = 730
HEIGHT = 530
CHANNELS = 3

BATCH_SIZE = 8
TRAIN_DATASET_NO = 3136

TEST_DATASET_NO = 348

SAMPLES_NO = TRAIN_DATASET_NO // BATCH_SIZE

DATA_PATH = "datasets/SUNRGB-D3D/SUNRGBD/kv2/kinect2data"
CSV_EXISTS = True


X_TRAIN_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_images_k2.csv'
y_TRAIN_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_annotations_k2.csv'
X_TEST_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_images_k2.csv'
y_TEST_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_annotations_k2.csv'
X_PREDICT_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/predict_k2.csv'

y_TRAIN_3DBB_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_annotations_3dbb_k2.csv'
y_TEST_3DBB_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_annotations_3dbb_k2.csv'


In [5]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
#
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree.

from typing import Tuple

import torch
import torch.nn.functional as F
from pytorch3d import _C
from torch.autograd import Function


# -------------------------------------------------- #
#                  CONSTANTS                         #
# -------------------------------------------------- #
"""
_box_planes and _box_triangles define the 4- and 3-connectivity
of the 8 box corners.
_box_planes gives the quad faces of the 3D box
_box_triangles gives the triangle faces of the 3D box
"""
_box_planes = [
    [0, 1, 2, 3],
    [3, 2, 6, 7],
    [0, 1, 5, 4],
    [0, 3, 7, 4],
    [1, 2, 6, 5],
    [4, 5, 6, 7],
]
_box_triangles = [
    [0, 1, 2],
    [0, 3, 2],
    [4, 5, 6],
    [4, 6, 7],
    [1, 5, 6],
    [1, 6, 2],
    [0, 4, 7],
    [0, 7, 3],
    [3, 2, 6],
    [3, 6, 7],
    [0, 1, 5],
    [0, 4, 5],
]


def _check_coplanar(boxes: torch.Tensor, eps: float = 1e-4) -> None:
    faces = torch.tensor(_box_planes, dtype=torch.int64, device=boxes.device)
    # pyre-fixme[16]: `boxes` has no attribute `index_select`.
    verts = boxes.index_select(index=faces.view(-1), dim=1)
    B = boxes.shape[0]
    P, V = faces.shape
    # (B, P, 4, 3) -> (B, P, 3)
    v0, v1, v2, v3 = verts.reshape(B, P, V, 3).unbind(2)

    # Compute the normal
    e0 = F.normalize(v1 - v0, dim=-1)
    e1 = F.normalize(v2 - v0, dim=-1)
    normal = F.normalize(torch.cross(e0, e1, dim=-1), dim=-1)

    # Check the fourth vertex is also on the same plane
    mat1 = (v3 - v0).view(B, 1, -1)  # (B, 1, P*3)
    mat2 = normal.view(B, -1, 1)  # (B, P*3, 1)
    if not (mat1.bmm(mat2).abs() < eps).all().item():
        msg = "Plane vertices are not coplanar"
        raise ValueError(msg)

    return


def _check_nonzero(boxes: torch.Tensor, eps: float = 1e-4) -> None:
    """
    Checks that the sides of the box have a non zero area
    """
    faces = torch.tensor(_box_triangles, dtype=torch.int64, device=boxes.device)
    # pyre-fixme[16]: `boxes` has no attribute `index_select`.
    verts = boxes.index_select(index=faces.view(-1), dim=1)
    B = boxes.shape[0]
    T, V = faces.shape
    # (B, T, 3, 3) -> (B, T, 3)
    v0, v1, v2 = verts.reshape(B, T, V, 3).unbind(2)

    normals = torch.cross(v1 - v0, v2 - v0, dim=-1)  # (B, T, 3)
    face_areas = normals.norm(dim=-1) / 2

    if (face_areas < eps).any().item():
        msg = "Planes have zero areas"
        raise ValueError(msg)

    return


class _box3d_overlap(Function):
    """
    Torch autograd Function wrapper for box3d_overlap C++/CUDA implementations.
    Backward is not supported.
    """

    @staticmethod
    def forward(ctx, boxes1, boxes2):
        """
        Arguments defintions the same as in the box3d_overlap function
        """
        vol, iou = _C.iou_box3d(boxes1, boxes2)
        return vol, iou

    @staticmethod
    def backward(ctx, grad_vol, grad_iou):
        raise ValueError("box3d_overlap backward is not supported")


def my_box3d_overlap(
    boxes1: torch.Tensor, boxes2: torch.Tensor, eps: float = 1e-4
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Computes the intersection of 3D boxes1 and boxes2.

    Inputs boxes1, boxes2 are tensors of shape (B, 8, 3)
    (where B doesn't have to be the same for boxes1 and boxes1),
    containing the 8 corners of the boxes, as follows:

        (4) +---------+. (5)
            | ` .     |  ` .
            | (0) +---+-----+ (1)
            |     |   |     |
        (7) +-----+---+. (6)|
            ` .   |     ` . |
            (3) ` +---------+ (2)


    NOTE: Throughout this implementation, we assume that boxes
    are defined by their 8 corners exactly in the order specified in the
    diagram above for the function to give correct results. In addition
    the vertices on each plane must be coplanar.
    As an alternative to the diagram, this is a unit bounding
    box which has the correct vertex ordering:

    box_corner_vertices = [
        [0, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [1, 1, 1],
        [0, 1, 1],
    ]

    Args:
        boxes1: tensor of shape (N, 8, 3) of the coordinates of the 1st boxes
        boxes2: tensor of shape (M, 8, 3) of the coordinates of the 2nd boxes
    Returns:
        vol: (N, M) tensor of the volume of the intersecting convex shapes
        iou: (N, M) tensor of the intersection over union which is
            defined as: `iou = vol / (vol1 + vol2 - vol)`
    """
    if not all((8, 3) == box.shape[1:] for box in [boxes1, boxes2]):
        raise ValueError("Each box in the batch must be of shape (8, 3)")

    # pyre-fixme[16]: `_box3d_overlap` has no attribute `apply`.
    vol, iou = _box3d_overlap.apply(boxes1, boxes2)

    return vol, iou


In [6]:
# 3D IoU Custom Metric

def iou_3D(y_true, y_pred):

    # Convert form tf tensor to numpy
    y_true_np = y_true.numpy().reshape(-1, 8, 2).astype(np.float32)
    y_pred_np = y_pred.numpy().reshape(-1, 8, 2).astype(np.float32)

    # Add the fictive third dimension
    y_true_np = np.append(y_true_np, np.zeros((y_true_np.shape[0], y_true_np.shape[1], 1)), axis=2)
    y_pred_np = np.append(y_pred_np, np.zeros((y_pred_np.shape[0], y_pred_np.shape[1], 1)), axis=2)


    # Reorder points to be compatible with pytorch3D requirements

    # TODO
    y_true_np_reorder = np.zeros(y_true_np.shape)
    y_pred_np_reorder = np.zeros(y_pred_np.shape)
    y_true_np_reorder[:, 0, :] = y_true_np[:, 2, :]
    y_pred_np_reorder[:, 0, :] = y_pred_np[:, 2, :]
    y_true_np_reorder[:, 1, :] = y_true_np[:, 3, :]
    y_pred_np_reorder[:, 1, :] = y_pred_np[:, 3, :]
    y_true_np_reorder[:, 2, :] = y_true_np[:, 7, :]
    y_pred_np_reorder[:, 2, :] = y_pred_np[:, 7, :]
    y_true_np_reorder[:, 3, :] = y_true_np[:, 6, :]
    y_pred_np_reorder[:, 3, :] = y_pred_np[:, 6, :]
    y_true_np_reorder[:, 4, :] = y_true_np[:, 1, :]
    y_pred_np_reorder[:, 4, :] = y_pred_np[:, 1, :]
    y_true_np_reorder[:, 5, :] = y_true_np[:, 0, :]
    y_pred_np_reorder[:, 5, :] = y_pred_np[:, 0, :]
    y_true_np_reorder[:, 6, :] = y_true_np[:, 4, :]
    y_pred_np_reorder[:, 6, :] = y_pred_np[:, 4, :]
    y_true_np_reorder[:, 7, :] = y_true_np[:, 5, :]
    y_pred_np_reorder[:, 7, :] = y_pred_np[:, 5, :]
    ''' 
            1 -------- 0
           /|         /|
          2 -------- 3 .
          | |        | |
          . 5 -------- 4
          |/         |/
          6 -------- 7
                |
                V
        (4) +---------+. (5)
            | ` .     |  ` .
            | (0) +---+-----+ (1)
            |     |   |     |
        (7) +-----+---+. (6)|
            ` .   |     ` . |
            (3) ` +---------+ (2)
    '''

    # Add the 1 value for "back" points
    y_true_np_reorder[:, 4:, :] = 1
    y_pred_np_reorder[:, 4:, :] = 1

    # Scale the value back to original dimensions
    y_true_np_reorder[:, :, 0] *= WIDTH
    y_true_np_reorder[:, :, 1] *= HEIGHT
    y_pred_np_reorder[:, :, 0] *= WIDTH
    y_pred_np_reorder[:, :, 1] *= HEIGHT

    # Convert from numpy to tf tensor
    y_true_torch_init = torch.tensor(y_true_np_reorder)
    y_pred_torch_init = torch.tensor(y_pred_np_reorder)

    # Reshape to (None, 8, 3) dimension
    y_true_torch_reshaped = torch.reshape(y_true_torch_init, (-1, 8, 3)).type(torch.float32)
    y_pred_torch_reshaped = torch.reshape(y_pred_torch_init, (-1, 8, 3)).type(torch.float32)


    # _, iou_3d = box3d_overal(torch_y_true, torch_y_pred)
    try:
        _, iou_3d = box3d_overlap(y_true_torch_reshaped, y_pred_torch_reshaped, 0.1)
    except ValueError:
        print('Got ValueError')
        iou_3d = 0

    return iou_3d
    # return 0


In [10]:
# 3D IoU Custom Metric

def iou_3D(y_true, y_pred):

    # Convert form tf tensor to numpy
    y_true_np = y_true.numpy().reshape(-1, 8, 2).astype(np.float32)
    y_pred_np = y_pred.numpy().reshape(-1, 8, 2).astype(np.float32)

    print(y_pred_np.shape)

    # Add the fictive third dimension
    y_true_np = np.append(y_true_np, np.zeros((y_true_np.shape[0], y_true_np.shape[1], 1)), axis=2)
    y_pred_np = np.append(y_pred_np, np.zeros((y_pred_np.shape[0], y_pred_np.shape[1], 1)), axis=2)


    # Reorder points to be compatible with pytorch3D requirements

    # TODO
    y_true_np_reorder = np.zeros(y_true_np.shape)
    y_pred_np_reorder = np.zeros(y_pred_np.shape)
    y_true_np_reorder[:, 0, :] = y_true_np[:, 2, :]
    y_pred_np_reorder[:, 0, :] = y_pred_np[:, 2, :]
    y_true_np_reorder[:, 1, :] = y_true_np[:, 3, :]
    y_pred_np_reorder[:, 1, :] = y_pred_np[:, 3, :]
    y_true_np_reorder[:, 2, :] = y_true_np[:, 7, :]
    y_pred_np_reorder[:, 2, :] = y_pred_np[:, 7, :]
    y_true_np_reorder[:, 3, :] = y_true_np[:, 6, :]
    y_pred_np_reorder[:, 3, :] = y_pred_np[:, 6, :]
    y_true_np_reorder[:, 4, :] = y_true_np[:, 1, :]
    y_pred_np_reorder[:, 4, :] = y_pred_np[:, 1, :]
    y_true_np_reorder[:, 5, :] = y_true_np[:, 0, :]
    y_pred_np_reorder[:, 5, :] = y_pred_np[:, 0, :]
    y_true_np_reorder[:, 6, :] = y_true_np[:, 4, :]
    y_pred_np_reorder[:, 6, :] = y_pred_np[:, 4, :]
    y_true_np_reorder[:, 7, :] = y_true_np[:, 5, :]
    y_pred_np_reorder[:, 7, :] = y_pred_np[:, 5, :]
    ''' 
            1 -------- 0
           /|         /|
          2 -------- 3 .
          | |        | |
          . 5 -------- 4
          |/         |/
          6 -------- 7
                |
                V
        (4) +---------+. (5)
            | ` .     |  ` .
            | (0) +---+-----+ (1)
            |     |   |     |
        (7) +-----+---+. (6)|
            ` .   |     ` . |
            (3) ` +---------+ (2)
    '''

    # Add the 1 value for "back" points
    y_true_np_reorder[:, 4:, :] = 1
    y_pred_np_reorder[:, 4:, :] = 1

    # Scale the value back to original dimensions
    y_true_np_reorder[:, :, 0] *= WIDTH
    y_true_np_reorder[:, :, 1] *= HEIGHT
    y_pred_np_reorder[:, :, 0] *= WIDTH
    y_pred_np_reorder[:, :, 1] *= HEIGHT

    # Convert from numpy to tf tensor
    y_true_torch_init = torch.tensor(y_true_np_reorder)
    y_pred_torch_init = torch.tensor(y_pred_np_reorder)

    # Reshape to (None, 8, 3) dimension
    y_true_torch_reshaped = torch.reshape(y_true_torch_init, (-1, 8, 3)).type(torch.float32)
    y_pred_torch_reshaped = torch.reshape(y_pred_torch_init, (-1, 8, 3)).type(torch.float32)


    # _, iou_3d = box3d_overal(torch_y_true, torch_y_pred)
    _, iou_3d = my_box3d_overlap(y_true_torch_reshaped, y_pred_torch_reshaped, 0.0)

    return iou_3d
    # return 0


In [7]:
def csv_lazy(file_name):
    with open(file_name) as f:
        r = csv.reader(f)
        
        for row in r:
            yield row

def batch_generator(X_file, batch_size):

    csv_generator = csv_lazy(X_file)

    while True:
        batch = np.array([])

        for i in range(batch_size):
            batch = np.append(batch, np.array(next(csv_generator)).astype('float') / 255.0)
        
        yield batch.reshape(batch_size, WIDTH, HEIGHT, CHANNELS).astype('float')

def label_generator(y_file, batch_size):
    csv_generator = csv_lazy(y_file)

    while True:
        batch = np.array([])

        for i in range(batch_size):
            batch = np.append(batch, next(csv_generator))
        
        yield batch.reshape(batch_size, 16).astype('float')
        
def model_generator(X_file, y_file, batch_size, limit_iteration):
    X_generator = batch_generator(X_file, batch_size)
    y_generator = label_generator(y_file, batch_size)
    
    iteration_no = 0

    while True:
        
        iteration_no += 1

        print('Iteration: ' + str(iteration_no))

        # Reset to start of file
        if iteration_no > limit_iteration:
            X_generator = batch_generator(X_file, batch_size)
            y_generator = label_generator(y_file, batch_size)
            iteration_no = 0

        X_batch = next(X_generator)
        y_batch = next(y_generator)

        yield (X_batch, y_batch)

In [11]:
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(16)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=[iou_3D], run_eagerly=True)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [12]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_3DBB_DATA_PATH, BATCH_SIZE, SAMPLES_NO // 10),
          steps_per_epoch = SAMPLES_NO // 10,
          epochs = 10,
          verbose = 2
          )

Iteration: 1
Epoch 1/10
Iteration: 2
(8, 8, 2)
Iteration: 3
(8, 8, 2)
Iteration: 4
(8, 8, 2)
Iteration: 5
(8, 8, 2)
Iteration: 6
(8, 8, 2)
Iteration: 7
(8, 8, 2)
Iteration: 8
(8, 8, 2)
Iteration: 9
(8, 8, 2)
Iteration: 10
(8, 8, 2)
Iteration: 11
(8, 8, 2)
Iteration: 12
(8, 8, 2)
Iteration: 13
(8, 8, 2)
Iteration: 14
(8, 8, 2)
Iteration: 15
(8, 8, 2)
Iteration: 16
(8, 8, 2)
Iteration: 17
(8, 8, 2)
Iteration: 18
(8, 8, 2)
Iteration: 19
(8, 8, 2)
Iteration: 20
(8, 8, 2)
Iteration: 21
(8, 8, 2)
Iteration: 22
(8, 8, 2)
Iteration: 23
(8, 8, 2)
Iteration: 24
(8, 8, 2)
Iteration: 25
(8, 8, 2)
Iteration: 26
(8, 8, 2)
Iteration: 27
(8, 8, 2)
Iteration: 28
(8, 8, 2)
Iteration: 29
(8, 8, 2)
Iteration: 30
(8, 8, 2)
Iteration: 31
(8, 8, 2)
Iteration: 32
(8, 8, 2)
Iteration: 33
(8, 8, 2)
Iteration: 34
(8, 8, 2)
Iteration: 35
(8, 8, 2)
Iteration: 36
(8, 8, 2)
Iteration: 37
(8, 8, 2)
Iteration: 38
(8, 8, 2)
Iteration: 39
(8, 8, 2)
Iteration: 40
(8, 8, 2)
39/39 - 259s - loss: 0.1137 - iou_3D: 0.3063 - 2

In [13]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_3DBB_DATA_PATH, 4, TEST_DATASET_NO // 4),
    verbose = 2,
    steps = TEST_DATASET_NO // 4 - 1
)

Iteration: 1
Iteration: 2
(4, 8, 2)
Iteration: 3
(4, 8, 2)
Iteration: 4
(4, 8, 2)
Iteration: 5
(4, 8, 2)
Iteration: 6
(4, 8, 2)
Iteration: 7
(4, 8, 2)
Iteration: 8
(4, 8, 2)
Iteration: 9
(4, 8, 2)
Iteration: 10
(4, 8, 2)
Iteration: 11
(4, 8, 2)
Iteration: 12
(4, 8, 2)
Iteration: 13
(4, 8, 2)
Iteration: 14
(4, 8, 2)
Iteration: 15
(4, 8, 2)
Iteration: 16
(4, 8, 2)
Iteration: 17
(4, 8, 2)
Iteration: 18
(4, 8, 2)
Iteration: 19
(4, 8, 2)
Iteration: 20
(4, 8, 2)
Iteration: 21
(4, 8, 2)
Iteration: 22
(4, 8, 2)
Iteration: 23
(4, 8, 2)
Iteration: 24
(4, 8, 2)
Iteration: 25
(4, 8, 2)
Iteration: 26
(4, 8, 2)
Iteration: 27
(4, 8, 2)
Iteration: 28
(4, 8, 2)
Iteration: 29
(4, 8, 2)
Iteration: 30
(4, 8, 2)
Iteration: 31
(4, 8, 2)
Iteration: 32
(4, 8, 2)
Iteration: 33
(4, 8, 2)
Iteration: 34
(4, 8, 2)
Iteration: 35
(4, 8, 2)
Iteration: 36
(4, 8, 2)
Iteration: 37
(4, 8, 2)
Iteration: 38
(4, 8, 2)
Iteration: 39
(4, 8, 2)
Iteration: 40
(4, 8, 2)
Iteration: 41
(4, 8, 2)
Iteration: 42
(4, 8, 2)
Iteration: 

[0.7206076979637146, 0.04186004772782326]

In [ ]:
model.predict(
    x = batch_generator(X_PREDICT_DATA_PATH, 10),
    verbose = 2,
    steps = 1
)

1/1 - 0s - 482ms/epoch - 482ms/step


array([[ 0.785371  , -1.1723423 ,  1.249822  ,  0.53392303,  0.3734668 ,
         0.4506517 , -0.4154103 , -0.5007538 , -0.4689716 , -0.8984044 ,
         1.5421717 ,  0.9015395 , -0.19124427,  1.4867687 ,  1.2176106 ,
         0.7547725 ],
       [ 0.531344  , -1.0514342 ,  1.0728439 ,  0.20657991, -0.09213046,
         0.42900223, -0.41997853, -0.47923175, -0.49751186, -0.61565775,
         1.5528492 ,  0.79883456, -0.12686267,  1.2853911 ,  1.094791  ,
         0.74612266],
       [ 0.6807431 , -1.1617111 ,  1.2317244 ,  0.44224158,  0.27666467,
         0.43158537, -0.41017064, -0.48017824, -0.49232978, -0.869902  ,
         1.5575064 ,  0.8534145 , -0.24310082,  1.3959389 ,  1.1862606 ,
         0.7783417 ],
       [ 0.6619082 , -1.1189016 ,  1.2156849 ,  0.43194294,  0.19547132,
         0.43393624, -0.43720636, -0.50657517, -0.46149462, -0.8338289 ,
         1.5862447 ,  0.8250378 , -0.17994805,  1.4317503 ,  1.1807278 ,
         0.77415615],
       [ 0.6584695 , -1.1616641 ,  1

In [ ]:
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(16)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=[iou_3D], run_eagerly=True)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_3DBB_DATA_PATH, BATCH_SIZE, SAMPLES_NO // 4),
          steps_per_epoch = SAMPLES_NO // 4,
          epochs = 10,
          verbose = 2
          )

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_3DBB_DATA_PATH, 4, TEST_DATASET_NO // 4),
    verbose = 2,
    steps = TEST_DATASET_NO // 4 - 1
)

In [ ]:
model.predict(
    x = batch_generator(X_PREDICT_DATA_PATH, 10),
    verbose = 2,
    steps = 1
)

In [ ]:
model.save('/content/drive/MyDrive/Colab Datasets/ResNet50-3DBB')

In [ ]:
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(16)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=[iou_3D], run_eagerly=True)

In [ ]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_3DBB_DATA_PATH, BATCH_SIZE, 390),
          steps_per_epoch = 390,
          epochs = 10,
          verbose = 2
          )

In [ ]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_3DBB_DATA_PATH, 4, 80),
    verbose = 2,
    steps = 80
)

In [ ]:
model.predict(
    x = batch_generator(X_PREDICT_DATA_PATH, 10),
    verbose = 2,
    steps = 1
)

In [ ]:
model.save('/content/drive/MyDrive/Colab Datasets/ResNet50-3DBB-fullbatch')